<a href="https://colab.research.google.com/github/dr-song-summer-project/ChatBot_NLP/blob/main/cosine_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install konlpy
!pip install xlrd
!pip install openpyxl

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 450kB 35.5MB/s 
     |████████████████████████████████| 92kB 9.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [8]:
#파일 업로드
from google.colab import files

#파일 읽기
#엑셀 및 txt 파일 읽기 후에 클래스 작성
uploaded = files.upload()

KeyboardInterrupt: ignored

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from tqdm.notebook import tqdm
from openpyxl import Workbook
import numpy as np
import pandas as pd
import copy


In [6]:
#TF IDF 를 이용한  Cosine 유사도 판별

class Similarity:

  ######################
  #  코사인 유사도 판별
  #  text : 질문
  #  questionList : DaraFrame의 질문목록
  #  excelDF : 읽어온 엑셀 DataFrame
  #######################

  def __init__(self, text, excelDF):
    self.text = text # 사용자 질문
    self.excelDF = excelDF
    self.setQuestion() #question 벡터화를 위한 question 리스트

  #question 초기화 함수
  def setQuestion(self):
    question = []
    df_np = pd.DataFrame.to_numpy(self.excelDF)
    for i in range(1, len(df_np)):
      question.append(df_np[i][0])
    self.questionList = question
    
    
  def vecotrization(self): #벡터화

    ## 후에 DB에서 해당 벡터요소를 가져오는 작업 추가 

    vectorizer = TfidfVectorizer() 
    TFIDF_matrix = copy.deepcopy(self.questionList) # 리스트 깊은 복사 수행
    TFIDF_matrix.append(self.text) # 리스트에 추가
    text_matrix = vectorizer.fit_transform(TFIDF_matrix) #벡터화 실시

    return text_matrix

  def get_recommendation(self):
    new_matrix = self.vecotrization() #새로운 메트릭스 생성
    cosine_sim = linear_kernel(new_matrix, new_matrix) # 코사인 유사도 계산

    # 수정 필요 ##

    #추가된 마지막 요소의 cosine_sim 의 list 가져온다
    sim_scores = list(enumerate(cosine_sim[-1]))

    # 유사도에 따라질문들을 정렬.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 질문의 인덱스
    test_ind = [i[0] for i in sim_scores]

    print(self.excelDF['질문'].iloc[test_ind])

    # 가장 유사한 10개의 답변을 리턴
    return self.excelDF['답변'].iloc[test_ind]


  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
def main():

  path = '/content/sample_data_processed_4600.xlsx'
  df = pd.read_excel(path)

  text = "피임약 복용중 여드름이 많이 나요"

  s = Similarity(text, df)
  a = s.get_recommendation()
  print(a) #답변 출력

if __name__ == '__main__':
  main()


4155     지금 고 1이고요 초경은 중 2 때 시작했습니다 최근 들어 생리주기가 너무 들쑥날...
4372                          차이가 뭐죠? 똑같은 증상에 먹어도 되는 약인가요
2609     피도 마니 나오고 빈혈과 두통까지 온다고 하는데 왜 이런 현상이 일어나는 지 궁금...
1353     피임약을 5개월 정도 복용하였습니다. 휴약기 4~5일 정도에 생리를 하였고, 36...
1240     피임약을 깜빡했는데 원래 수요일에 먹어야 한다면 이틀 뒤에 먹으면 약 효과과 없나...
4455     색깔과 모양은 모두 같은데 제가 먹는 약은 회색 부분(SRM) 빨간 부분(HT) ...
382      하루도 빠짐없이 잘 복용했다면 계속 피임 효과가 있는 거죠? 첫 번째 팩 51-5...
990      경구피임약은 두 번째 복용 중이고 이번에 생리 전이 아니고 생리 끝난 후 2일 뒤...
4235     올해 37살입니다 주부 고요 가슴이 콕콕 쑤시면서 아프네요 가슴이랑 겨드랑이 아파...
3765     * 개인정보보호를 위하여 상담 글에 실명을 사용하지 마십시오. 주요 증상, 의 심...
Name: 질문, dtype: object
4155     중학생 때는 생리를 제대로 했는데 최근 들어 생리가 점점 늦어지는 거라면 한 번 ...
4372     같은 약입니다. 알약이나 캡슐처럼 고체화시킨 약은 아목시 실린 또는 아목시 실린 ...
2609     응급피임약의 부작용으로 생각됩니다. 조금 기 다려 보시다가 더 지속되거나 양이 많...
1353     유방통증이나 아랫배가 아픈 느낌만으로는 임 신임을 진단하거나 의심할 만한 확실한 ...
1240     피임약을 시작한 지 일주일 이내인지 2주가 지났는 여부에 따라 달라집니다. 시작한...
4455     똑같은 약입니다. 아마 시간이 지나면서 약을 감싸는 캡슐 색이 바뀐 것 같습니다....
382      일단 피임약을 정확하게 잘 복용하고 계신 것을 칭찬드리고 싶습니다. 날

***피임약과 관련없는 데이터 삭제*** (1) : np array에서 삭제


In [10]:
#filtering

#엑셀 파일 중 피임약 포함 안되는 Q&A 삭제, 새로운 엑셀 만들어서 다시 확인
path = '/content/sample_data_processed_4600.xlsx'


#Excel 읽고 numpy array로 변경
excel = pd.read_excel(path)
excelNP = pd.DataFrame.to_numpy(excel)

#피임약과 관련된 단어
birthPillWord = ['피임약', '머시론', '야즈', '야스민', '피임 약', '다온', '센스리베', '센스 리베', '멜리안', '마이보라', '경구피임', '엘라원', '머 시론'
                , '피암', '노레보', '엘리원', '피임목적', '엘라', '경구 피임제', '머*론', '트*페린', '엘*원', '미뉴렛', '미 뉴렛', '머시', '노레*'
                , '센스데이', '센스 데이'
                ]

WrongQandA = [] # 피임약이 존재하지 않는 리스트
newExcelNP = [] # 피임약이 존재하는 리스트


for i in tqdm(range(len(excelNP))):
  
  if excelNP[i] is None : break #test

  #질문에 피임약에 관련된 단어들 제외
  if any(ele in excelNP[i][0] for ele in birthPillWord):
    newExcelNP.append(excelNP[i])
    continue
  #답변에 피임약 관련된 단어들 존재할 시 제외 
  elif any(ele in excelNP[i][1] for ele in birthPillWord):
    newExcelNP.append(excelNP[i])
    continue
  else:
    WrongQandA.append(excelNP[i])
    

 

***피임약과 관련없는 데이터 삭제*** (2) : 엑셀에 저장

In [11]:
#index 82, index 176 (답변 작성 x), 

#br 도 빼기



def makeXlsxFile(path, name, list):
  wb = Workbook()
  filename = path + name

  sheet = wb.active
  sheet.append(['질문', '답변'])
  for x in tqdm(list):
    sheet.append([x[0], x[1]])
    wb.save(filename)

path = '/content/'
name = 'newxlsx.xlsx'

makeXlsxFile(path, name, newExcelNP)